### Project Pipeline

Workflow:
1. common load pipeline
2. train/test split - in order to avoid stat leaks we split at the beginning
3. ML/DL pipelines
4. Model fit -> other notebook

In [2]:
import os
import sys

sys.dont_write_bytecode = True
root_dir = os.path.abspath(os.pardir)
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [3]:
import requests
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from configs.constants import *
from scripts.data import *
from scripts.features import *
from scripts.preprocessing import *
from scripts.pipelines import *

configuration

In [4]:
split_random_state = TTS_SEED
split_test_size = TEST_SIZE

In [5]:
metafile = '../data/results/complete_metadata_mapping_2.csv'
data_dir_folder_path = os.path.abspath(os.path.join(root_dir, DATA_DIR_FOLDER))
data_dir_path = os.path.abspath(os.path.join(data_dir_folder_path, DATA_DIR))

**1. common load pipeline**

In [6]:
run_ = False
if run_:
    rows = []
    for hea in iter_header_paths(data_dir_folder_path):
        rows.append(parse_header(hea))

    meta_df = pd.DataFrame(rows)
    print("Number of records:", len(meta_df))
    print(meta_df.head())

In [7]:
meta_df = pd.read_csv(metafile)
meta_df['dx_codes'] = meta_df['dx_codes'].map(json.loads)

**2. train/test split**

In [8]:
X = meta_df.drop('dx_codes', axis=1)
y = meta_df['dx_codes']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=split_random_state)

**3. ML/DL pipeline**

- TODO: denoising -> nk.ecg_clean
- TODO: signal filter

target encoding

ML pipeline

In [18]:
dirpath = '../data/preprocessing/ML'

In [21]:
run_ = False
filter_mode = 'bandpass'

In [20]:
if run_:
    rows, logs = load_meta_df_stream(X_train, 
                                    filter_mode=filter_mode,
                                    save_to_file=True,
                                    dirpath=dirpath,
                                    filename='meta_df_preprocess_X_train_1.csv')

    rows, logs = load_meta_df_stream(X_test, 
                                    filter_mode=filter_mode,
                                    save_to_file=True,
                                    dirpath=dirpath,
                                    filename='meta_df_preprocess_X_test_1.csv')

In [16]:
save_target_ = False

In [17]:
if save_target_:
    y_train_file = 'y_train.csv'
    y_test_file = 'y_test.csv'
    y_train.to_csv(os.path.join(dirpath, y_train_file))
    y_test.to_csv(os.path.join(dirpath, y_test_file))

DL pipeline

**4. Model fit**

notes:
- we are fitting on snomed CT codes -> for diagnosis we need mapping `snomed CT -> condition`

**full pipeline**